In [3]:
#importing libraries
import cv2
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from keras import backend as k

In [4]:
# Mount Google Drive
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

# Navigate to the path where the zip file is located
zip_file_path = '/content/drive/MyDrive/medical-mnist.zip'

import os
import zipfile

# Extract zip file from Google Drive to Colab
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall('/content/dataset')  # Extract to /content/dataset folder

# List the contents of the extracted folder
os.listdir('/content/dataset')

# Define the dataset directory
directory = "/content/dataset"

Mounted at /content/drive


# Loading the Dataset

In [5]:

train_ds, test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory, # Directory where the data is located
    labels='inferred', # labels are generated from the directory structure
    label_mode='categorical', #labels are encoded as a categorical vector
    class_names=None, #explicit list of class names in alphabetical order
    color_mode='grayscale', #images will be converted to have 1 channel
    batch_size=100, #Size of the batches of data.
    image_size=(48, 48), #Size to resize images to after they are read (h, w)
    shuffle=True, #Whether to shuffle the data.
    seed=42, #Optional random seed for shuffling and transformations.
    validation_split=0.2, #0<float<1, fraction of data to reserve for cv
    subset='both', #returns a tuple of two datasets (training & cv)
    interpolation='bilinear', #interpolation method used when resizing images
    follow_links=False, #Whether to visit subdirectories pointed to by symlinks
    crop_to_aspect_ratio=False, #resize the images with aspect ratio distortion
    pad_to_aspect_ratio=True, #resize the images with aspect ratio distortion
    data_format=None, #If None uses keras.config.image_data_format()
    verbose=True #Whether to display no. info on classes & no. of files found
)

Found 58954 files belonging to 6 classes.
Using 47164 files for training.
Using 11790 files for validation.


In [6]:
#Displays the 6 class names
print(train_ds.class_names)
#Displays element specifications
train_ds

['AbdomenCT', 'BreastMRI', 'CXR', 'ChestCT', 'Hand', 'HeadCT']


<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 48, 48, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None, 6), dtype=tf.float32, name=None))>

In [7]:
#convert TensorFlow Datasets into NumPy format
train_ds_np = tfds.as_numpy(train_ds)
test_ds_np = tfds.as_numpy(test_ds)

In [8]:
#X_train: Shape: [1, 48, 48, 1] Represents a single grayscale image of zeros size 48x48 for training (the last dimension 1 indicates a single color channel for grayscale).
#y_train: Shape: [1, 6] Represents the one-hot encoded label of zeros for the training data, assuming there are 6 classes.
#X_test: Shape: [1, 48, 48, 1] Represents a single grayscale image of zeros size 48x48 for testing.
#y_test: Shape: [1, 6] Represents the one-hot encoded label of zeros for the testing data.
X_train, y_train, X_test, y_test = np.zeros([1, 48, 48, 1]), np.zeros([1, 6]), np.zeros([1, 48, 48, 1]), np.zeros([1, 6])

#The array has a bactch size 32 images, so reshaping it as [-1, 48, 48, 1] will keep the batch size the same, and the shape will remain unchanged.
#This would be equivalent to the original shape of (32, 48, 48, 1).
for ele in train_ds_np:
    #print(f"Shape of ele[0] at index {i}: {ele[0].shape}")
    X_train = np.append(X_train, ele[0].reshape([-1,48,48,1]), axis=0)
    #print(f"Shape of ele[1] at index {i}: {ele[1].shape}")
    y_train = np.append(y_train, ele[1].reshape([-1,6]), axis=0)

for ele in test_ds_np:
    #print(f"Shape of ele[0] at index {i}: {ele[0].shape}")
    X_test = np.append(X_test, ele[0].reshape([-1,48,48,1]), axis=0)
    #print(f"Shape of ele[1] at index {i}: {ele[1].shape}")
    y_test = np.append(y_test, ele[1].reshape([-1,6]), axis=0)

#omit the first grayscale 48x48 image of zeros
X_train, y_train, X_test, y_test = X_train[1:,:,:,:], y_train[1:,:], X_test[1:,:,:,:], y_test[1:,:]

#printing shapes of training and testing sets
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

#printing the labels of the 421st image in train and the 100th image in test
y_train[421], y_test[100]

(47164, 48, 48, 1) (47164, 6) (11790, 48, 48, 1) (11790, 6)


(array([0., 1., 0., 0., 0., 0.]), array([0., 0., 0., 0., 1., 0.]))

# Data Preprocessing:

In [9]:
#normalizing the image data by converting the pixel values from the range [0, 255] (which is typical for 8-bit grayscale images) to the range [0, 1]
X_train_normalized = X_train.astype('float32')
X_test_normalized = X_test.astype('float32')
X_train_normalized /= 255.0
X_test_normalized /= 255.0

# Model-1: CNN

In [10]:
import keras
from tensorflow.keras import datasets, layers, models, optimizers

In [11]:
my_model = models.Sequential([
    #2D Convolutional Layer with 32 filters each of size 3x3
    #Shape of the input images are 48x48 pixels with 1 channel
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)),
    #Max Pooling Layer with a 2x2 pool size
    layers.MaxPooling2D((2, 2)),
    #2D Convolutional Layer with 64 filters each of size 3x3
    layers.Conv2D(64, (3, 3), activation='relu'),
    #Max Pooling Layer with a 2x2 pool size
    layers.MaxPooling2D((2, 2)),
    #2D Convolutional Layer with 64 filters each of size 3x3
    #This Layer flattens the 3D output of the last Conv2D layer into 1D array
    layers.Flatten(),
    #A fully connected (Dense) layer with 64 neurons
    layers.Dense(64, activation='relu'),
    #Dense layer with 64 neurons corresponding to the 6 classes
    layers.Dense(6, activation='softmax')
])

my_model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 46, 46, 32)          │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 23, 23, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 21, 21, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 10, 10, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 6400)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │         409,664 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 6)                   │             390 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 428,870 (1.64 MB)

 Trainable params: 428,870 (1.64 MB)

 Non-trainable params: 0 (0.00 B)

In [12]:
my_model.compile(
    optimizer = 'adam', #the learning rate defaults to 0.001
    loss = 'categorical_crossentropy', #use this cross entropy function where are 2+ label classes in one-hot representation
    #loss_weights=None,
    metrics=['accuracy']
    #weighted_metrics=None,
    #run_eagerly=False,
    #steps_per_execution=1,
    #jit_compile='auto',
    #auto_scale_loss=True
)

In [13]:
#printing shapes and data types of training and testing sets
print(X_train_normalized.shape, y_train.shape, X_test.shape, y_test.shape)
print(X_train_normalized.dtype, y_train.dtype, X_test.dtype, y_test.dtype)

(47164, 48, 48, 1) (47164, 6) (11790, 48, 48, 1) (11790, 6)
float32 float64 float64 float64


In [14]:
history = my_model.fit(
    x=X_train_normalized,
    y=y_train,
    batch_size=None,
    epochs=10,
    #verbose=2,
    #callbacks=None,
    validation_split=0.2,
    #validation_data = (X_test,y_test)
    shuffle=True,
    #class_weight=None,
    #sample_weight=None,
    #initial_epoch=0,
    #steps_per_epoch=None,
    #validation_steps=None,
    #validation_batch_size=None,
    #validation_freq=1
)

Epoch 1/10
1180/1180 ━━━━━━━━━━━━━━━━━━━━ 116s 97ms/step - accuracy: 0.9550 - loss: 0.1459 - val_accuracy: 0.9976 - val_loss: 0.0108
Epoch 2/10
1180/1180 ━━━━━━━━━━━━━━━━━━━━ 142s 97ms/step - accuracy: 0.9985 - loss: 0.0054 - val_accuracy: 0.9978 - val_loss: 0.0085
Epoch 3/10
1180/1180 ━━━━━━━━━━━━━━━━━━━━ 112s 95ms/step - accuracy: 0.9991 - loss: 0.0030 - val_accuracy: 0.9985 - val_loss: 0.0052
Epoch 4/10
1180/1180 ━━━━━━━━━━━━━━━━━━━━ 143s 96ms/step - accuracy: 0.9996 - loss: 0.0015 - val_accuracy: 0.9969 - val_loss: 0.0159
Epoch 5/10
1180/1180 ━━━━━━━━━━━━━━━━━━━━ 139s 93ms/step - accuracy: 0.9985 - loss: 0.0046 - val_accuracy: 0.9979 - val_loss: 0.0075
Epoch 6/10
1180/1180 ━━━━━━━━━━━━━━━━━━━━ 145s 96ms/step - accuracy: 0.9995 - loss: 9.4942e-04 - val_accuracy: 0.9989 - val_loss: 0.0041
Epoch 7/10
1180/1180 ━━━━━━━━━━━━━━━━━━━━ 110s 93ms/step - accuracy: 0.9992 - loss: 0.0028 - val_accuracy: 0.9984 - val_loss: 0.0071
Epoch 8/10
1180/1180 ━━━━━━━━━━━━━━━━━━━━ 109s 93ms/step - accura

In [15]:
# Evaluate the model
test_loss, test_acc = my_model.evaluate(X_test_normalized, y_test)
print('Test accuracy:', test_acc)

369/369 ━━━━━━━━━━━━━━━━━━━━ 10s 27ms/step - accuracy: 0.9986 - loss: 0.0043
Test accuracy: 0.9985581040382385


Printing some predictions:

In [16]:
predictions = my_model.predict(X_test_normalized[:10])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step


In [17]:
predicted_labels = predictions.argmax(axis=1)
actual_labels = y_test.argmax(axis=1)

print(f"Predicted Labels: {predicted_labels}")

print(f"Actual Labels: {actual_labels[:10]}")

Predicted Labels: [1 1 2 2 0 0 3 3 5 3]
Actual Labels: [1 1 2 2 0 0 3 3 5 3]


Model 2

In [18]:
my_model = models.Sequential([
    #2D Convolutional Layer with 32 filters each of size 3x3
    #Shape of the input images are 48x48 pixels with 1 channel
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)),
    #Max Pooling Layer with a 2x2 pool size
    layers.MaxPooling2D((2, 2)),
    #2D Convolutional Layer with 64 filters each of size 3x3
    layers.Conv2D(64, (3, 3), activation='relu'),
    #Max Pooling Layer with a 2x2 pool size
    layers.Flatten(),
    #A fully connected (Dense) layer with 64 neurons
    layers.Dense(64, activation='relu'),
    #Dense layer with 64 neurons corresponding to the 6 classes
    layers.Dense(6, activation='softmax')
])

my_model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)                    │ (None, 46, 46, 32)          │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 23, 23, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 21, 21, 32)          │           9,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 10, 10, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 3200)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 64)                  │         204,864 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 6)                   │             390 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 214,822 (839.15 KB)

 Trainable params: 214,822 (839.15 KB)

 Non-trainable params: 0 (0.00 B)

In [19]:
my_model.compile(
    optimizer = 'adam', #the learning rate defaults to 0.001
    loss = 'categorical_crossentropy', #use this cross entropy function where are 2+ label classes in one-hot representation
    #loss_weights=None,
    metrics=['accuracy']
    #weighted_metrics=None,
    #run_eagerly=False,
    #steps_per_execution=1,
    #jit_compile='auto',
    #auto_scale_loss=True
)

In [20]:
history = my_model.fit(
    x=X_train_normalized,
    y=y_train,
    batch_size=100,
    epochs=10,
    #verbose=2,
    #callbacks=None,
    validation_split=0.2,
    #validation_data = (X_test,y_test)
    shuffle=True,
    #class_weight=None,
    #sample_weight=None,
    #initial_epoch=0,
    #steps_per_epoch=None,
    #validation_steps=None,
    #validation_batch_size=None,
    #validation_freq=1
)

Epoch 1/10
378/378 ━━━━━━━━━━━━━━━━━━━━ 89s 231ms/step - accuracy: 0.9174 - loss: 0.3091 - val_accuracy: 0.9942 - val_loss: 0.0305
Epoch 2/10
378/378 ━━━━━━━━━━━━━━━━━━━━ 136s 215ms/step - accuracy: 0.9944 - loss: 0.0248 - val_accuracy: 0.9981 - val_loss: 0.0118
Epoch 3/10
378/378 ━━━━━━━━━━━━━━━━━━━━ 82s 214ms/step - accuracy: 0.9973 - loss: 0.0090 - val_accuracy: 0.9977 - val_loss: 0.0078
Epoch 4/10
378/378 ━━━━━━━━━━━━━━━━━━━━ 81s 213ms/step - accuracy: 0.9991 - loss: 0.0039 - val_accuracy: 0.9972 - val_loss: 0.0096
Epoch 5/10
378/378 ━━━━━━━━━━━━━━━━━━━━ 82s 214ms/step - accuracy: 0.9988 - loss: 0.0033 - val_accuracy: 0.9963 - val_loss: 0.0130
Epoch 6/10
378/378 ━━━━━━━━━━━━━━━━━━━━ 81s 211ms/step - accuracy: 0.9994 - loss: 0.0019 - val_accuracy: 0.9986 - val_loss: 0.0057
Epoch 7/10
378/378 ━━━━━━━━━━━━━━━━━━━━ 80s 212ms/step - accuracy: 0.9998 - loss: 6.2995e-04 - val_accuracy: 0.9987 - val_loss: 0.0057
Epoch 8/10
378/378 ━━━━━━━━━━━━━━━━━━━━ 84s 218ms/step - accuracy: 0.9997 - lo

In [21]:
# Evaluate the model
test_loss, test_acc = my_model.evaluate(X_test_normalized, y_test)
print('Test accuracy:', test_acc)

369/369 ━━━━━━━━━━━━━━━━━━━━ 8s 23ms/step - accuracy: 0.9992 - loss: 0.0026
Test accuracy: 0.9988973736763


In [22]:
predictions = my_model.predict(X_test_normalized[:10])

predicted_labels = predictions.argmax(axis=1)
actual_labels = y_test.argmax(axis=1)

print(f"Predicted Labels: {predicted_labels}")

print(f"Actual Labels: {actual_labels[:10]}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
Predicted Labels: [1 1 2 2 0 0 3 3 5 3]
Actual Labels: [1 1 2 2 0 0 3 3 5 3]
